# Requierments

In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, minimize, SolverFactory
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore') 

# Parameters and Variables

In [3]:
model =pyo.ConcreteModel()
model.iset=pyo.Set(initialize=range(1,4))
model.sset=pyo.Set(initialize=range(1,11))
model.C=pyo.Param(initialize=1000)
model.Q=pyo.Param(initialize=0.02)
model.s=pyo.Param(model.sset, default=0, initialize=0.1)

model.x=pyo.Var(model.iset, domain=pyo.NonNegativeReals)
df = pd.read_excel('Book1.xlsx')
df_new=df.copy()
df_new.index= range(1,11)
df_new

,1,2,3
1,0.05,0.10,-0.05
2,-0.05,0.10,0.15
3,0.10,0.10,0.20
4,-0.05,-0.15,0.10
5,0.07,0.05,-0.20
6,-0.10,0.20,0.10
7,0.15,0.05,-0.10
8,-0.15,-0.10,-0.10
9,0.05,0.10,0.05
10,0.05,0.05,0.05


In [4]:
data_dict = {}
for i, row in df_new.iterrows():
    for j, value in row.items():
        data_dict[(i, j)] = value
#تعریف آرایه برای مقادیر a 
model.a=pyo.Param(model.sset, model.iset, default=0, initialize=data_dict)


In [5]:
data_array = np.zeros((10,3))
for (x, y), value in data_dict.items():
    data_array[x-1, y-1] = value

#محاسبه میانگین
mean = np.mean(data_array, axis=0)
mean_df = pd.DataFrame(mean, columns=['Mean'], index=[1,2,3])
mean_df

,Mean
1,0.012
2,0.050
3,0.020


In [6]:
def rule_mean(model, i):
 return mean[i-1]
model.mean=pyo.Param(model.iset, initialize=rule_mean)

# Objective

In [7]:
model.min_sum = Var(domain=pyo.Reals)
def min_sum_constraint_rule(model, s):
    return model.min_sum <= sum(model.a[s, i] * model.x[i] for i in model.iset)
model.min_sum_constraints = Constraint(model.sset, rule=min_sum_constraint_rule)

In [8]:
model.obj = Objective(expr=(1/model.C)*(model.min_sum), sense=pyo.maximize)

In [9]:
def rule_const1(model):
 return sum(model.mean[i]*model.x[i] for i in model.iset) >= model.Q * model.C
model.const1 = pyo.Constraint(rule=rule_const1)

def rule_const2(model):
 return sum(model.x[i] for i in model.iset) == model.C
model.const2 = pyo.Constraint(rule=rule_const2)


In [10]:
%reload_ext gams.magic

In [11]:
%gams set i(*);


In [12]:
%%gams
set j;
parameter p(i,j);
parameter p2(i,j);


In [13]:
solver = SolverFactory('gams')
result = solver.solve(model, solver='BARON', keepfiles=True, tee=True, io_options={'solvelink': 2},add_options=['option optcr=0;'])
model.display()

x1_value = pyo.value(model.x[1])
x2_value = pyo.value(model.x[2])
x3_value=pyo.value(model.x[3])
objective_value = pyo.value(model.obj)

print(f"x1 = {x1_value}")
print(f"x2 = {x2_value}")
print(f"x3 = {x3_value}")

--- Job model.gms Start 07/29/24 12:48:00 47.3.0 125a0e22 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\47\gmsprmNT.txt
--- GAMS Parameters defined
    Input C:\Users\farim\AppData\Local\Temp\tmp1p_gmg8c\model.gms
    Output C:\Users\farim\AppData\Local\Temp\tmp1p_gmg8c\output.lst
    ScrDir C:\Users\farim\AppData\Local\Temp\tmp1p_gmg8c\225a\
    SysDir C:\GAMS\47\
    CurDir C:\Users\farim\AppData\Local\Temp\tmp1p_gmg8c\
    LogOption 3
Licensee: GAMS Demo, for EULA and demo limitations see   G240530/0001CB-GEN
          https://www.gams.com/latest/docs/UG%5FLicense.html         DC0000
          C:\GAMS\47\gamslice.txt
          Demo license for demonstration and instructional purposes only
Processor information: 1 socket(s), 4 core(s), and 8 thread(s) available
GAMS 47.3.0   Copyright (C) 1987-2024 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(128) 2 Mb
--- Starting execution: elapsed 0:00:00.011
--- model.gms(53) 3 Mb
--- Generating LP model G